In [209]:
from bs4 import BeautifulSoup
import pandas as pd
from IPython.core.display import display, HTML
import requests
import re 
import pickle

In [449]:
# Pulling the table from the page of 

urlsG_1 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=G'
urlsG_2 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=G&offset=200'
urlsPG_1 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG'
urlsPG_2 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG&offset=200'
urlsPG_3 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?offset=400&by_mpaa=PG'
urlsPG_4 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG&offset=600'
urlsPG_5 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?offset=800&by_mpaa=PG'
urlsPG13_1 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13'
urlsPG13_2 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=200'
urlsPG13_3 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?offset=400&by_mpaa=PG-13'
urlsPG13_4 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=600'
urlsPG13_5 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=800'
urlsR_1 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=R'
urlsR_2 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=R&offset=200'
urlsR_3 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=R&offset=400'
urlsR_4 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?offset=600&by_mpaa=R'
urlsR_5 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=R&offset=800'


response = requests.get(urlsR_1)

response.status_code

200

In [450]:
page = response.text
soup = BeautifulSoup(page)
table = soup.find('table')

In [451]:
rows = [row for row in table.find_all('tr')]

Pulled data from the webpage and looped through the webpage script to capture the list of url links

In [472]:
film_links = {}


for row in rows[1:201]:
    items = row.find_all('td')
    link = items[0].find('a')
    title, url = link.text, link['href']
    film_links[title] = [url]
    
film_links

{'The Passion of the Christ': ['/title/tt0335345/?ref_=bo_cso_table_1'],
 'Deadpool': ['/title/tt1431045/?ref_=bo_cso_table_2'],
 'American Sniper': ['/title/tt2179136/?ref_=bo_cso_table_3'],
 'Joker': ['/title/tt7286456/?ref_=bo_cso_table_4'],
 'It': ['/title/tt1396484/?ref_=bo_cso_table_5'],
 'Deadpool 2': ['/title/tt5463162/?ref_=bo_cso_table_6'],
 'The Matrix Reloaded': ['/title/tt0234215/?ref_=bo_cso_table_7'],
 'The Hangover': ['/title/tt1119646/?ref_=bo_cso_table_8'],
 'The Hangover Part II': ['/title/tt1411697/?ref_=bo_cso_table_9'],
 'Beverly Hills Cop': ['/title/tt0086960/?ref_=bo_cso_table_10'],
 'The Exorcist': ['/title/tt0070047/?ref_=bo_cso_table_11'],
 'Logan': ['/title/tt3315342/?ref_=bo_cso_table_12'],
 'Ted': ['/title/tt1637725/?ref_=bo_cso_table_13'],
 'Saving Private Ryan': ['/title/tt0120815/?ref_=bo_cso_table_14'],
 'A Star Is Born': ['/title/tt1517451/?ref_=bo_cso_table_15'],
 'It Chapter Two': ['/title/tt7349950/?ref_=bo_cso_table_16'],
 '300': ['/title/tt041644

In [516]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [517]:
import dateutil.parser

def money_to_int(moneystring):
'''
Cleans box office revenue and converts the data type to integer

INPUT: 'string with curreny notation'
OUTPUT: 'integer stripped of curreny notation'
'''    
    
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

# Removed Genre due to volume of missing data from Box Office Mojo; not enough time to reincorporate the new data and analyze a smaller dataset. 
# def clean_genre(genres):
#     try:
#         genre = genres.split()
#         return genre
#     except:
#         return None

def dist_cleaner(dist_name):
    '''
    Cleans distributor information pulled from webpage
    
    INPUT: 
    OUTPUT:
    '''
    try:
        distributor = dist_name.split(' ')
        del distributor[:-4:-1]
        distributor = ' '.join(distributor)
        distributor = distributor.strip('See')
        return distributor
    except:
        return None

def runtime_to_minutes(runtimestring):
    '''
    Cleans runtime string converts the units of time to minutes and returns it as an integer
    INPUT: String
    OUTPUT Integer
    '''
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    '''
    
    '''
    date = dateutil.parser.parse(datestring)
    return date

In [518]:
def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - distributor
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    # removed    - genre
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    
    headers = ['movie_title', 'distributor', 'domestic_total_gross',
               'runtime_minutes', 'rating', 'release_date']
    
    #Get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()

    #Get distributor
    distributor_string = get_movie_value(soup, 'Distributor')
    distributor = dist_cleaner(distributor_string)
    
    #Get domestic gross
    raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                    .find_all('span', class_='money')[0]
                                    .text
                               )
    domestic_total_gross = money_to_int(raw_domestic_total_gross)

    #Get runtime
    raw_runtime = get_movie_value(soup,'Running')
    runtime = runtime_to_minutes(raw_runtime)
    
    #Get rating
    rating = get_movie_value(soup,'MPAA')

    #Get release date
    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_date = to_date(raw_release_date)
    
# Removed due to volume of missing data points from genre    
#     #Get genre
#     raw_genre = get_movie_value(soup, 'Genre')
#     genre = clean_genre(raw_genre)
    
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title,
                                distributor,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date]))

    return movie_dict

In [523]:
films = pd.DataFrame(film_links).T  
films.columns = ['link_stub']

films.head

pandas.core.frame.DataFrame

In [520]:
#Runs transposed dataframe through a loop and function get_movie_dict attaching the beginning of the url and grabbing features listed in the function get_movie_dict

film_info_list = []

for link in films.link_stub:
    film_info_list.append(get_movie_dict(link))

In [521]:
film_info = pd.DataFrame(film_info_list)  #convert list of dict to df
film_info.set_index('movie_title', inplace=True)

R1 = film_info

In [513]:
R1

,distributor,domestic_total_gross,runtime_minutes,rating,release_date,genre
movie_title,,,,,,
The Passion of the Christ,Newmarket Films,370782930,127.0,R,2004-02-25,[]
Deadpool,Twentieth Century Fox,363070709,108.0,R,2016-02-09,[]
American Sniper,Warner Bros.,350126372,133.0,R,2014-12-25,[]
Joker,Warner Bros.,335451311,122.0,R,2019-10-02,[]
It,Warner Bros.,328828874,135.0,R,2017-09-06,[]
...,...,...,...,...,...,...
Crimson Tide,Walt Disney Studios Motion Pictures,91387195,116.0,R,1995-05-12,[]
"Me, Myself & Irene",Twentieth Century Fox,90570999,116.0,R,2000-06-23,[]
Get Hard,Warner Bros.,90411453,100.0,R,2015-03-26,"[Comedy, Crime]"


Below is a list of cells run to capture and fill in the rating of films for each, they were repeated for each subset of the table scraped. 

In [311]:
#G_rated_Films = pd.concat([G1, G2])

In [312]:
#G_rated_Films.fillna('G', inplace=True)

In [314]:
#with open('my_film_data_G.pkl', 'wb') as picklefile:
#   pickle.dump(G_rated_Films, picklefile)

In [316]:
#PG_rated_Films = pd.concat([PG1, PG2, PG3, PG4, PG5])

In [317]:
#PG_rated_Films.fillna('PG', inplace=True)

In [319]:
#with open('my_film_data_PG.pkl', 'wb') as picklefile:
#    pickle.dump(PG_rated_Films, picklefile)

In [381]:
#PG13_rated_Films = pd.concat([PG131, PG132, PG133, PG134, PG135])

In [382]:
#PG13_rated_Films.fillna('PG-13', inplace=True)

In [384]:
#with open('my_film_data_PG13.pkl', 'wb') as picklefile:
#    pickle.dump(PG13_rated_Films, picklefile)

In [463]:
#R_rated_Films = pd.concat([R1, R2, R3, R4, R5])

In [464]:
#R_rated_Films.fillna('R', inplace=True)

In [465]:
#with open('my_film_data_R.pkl', 'wb') as picklefile:
#    pickle.dump(R_rated_Films, picklefile)

In [466]:
ls

Box-Office Mojo Script.ipynb  my_film_data_PG.pkl
Web-Scraping.ipynb            my_film_data_PG13.pkl
my_film_data.pkl              my_film_data_R.pkl
my_film_data_G.pkl


In [468]:
AllRatings = pd.concat([G_rated_Films, PG_rated_Films, PG13_rated_Films, R_rated_Films ])

In [469]:
AllRatings

,distributor,domestic_total_gross,runtime_minutes,rating,release_date
movie_title,,,,,
Toy Story 4,Walt Disney Studios Motion Pictures,434038008,100,G,2019-06-20
The Lion King,Walt Disney Studios Motion Pictures,422783777,88,G,1994-06-15
Toy Story 3,Walt Disney Studios Motion Pictures,415004880,103,G,2010-06-16
Finding Nemo,Walt Disney Studios Motion Pictures,380843261,100,G,2003-05-30
"Monsters, Inc.",Walt Disney Studios Motion Pictures,290642256,92,G,2001-11-02
...,...,...,...,...,...
Three Days of the Condor,Paramount Pictures,27476252,117,R,1975-09-24
Friday,New Line Cinema,27467564,91,R,1995-04-28
The Brothers,creen Gems,27457409,106,R,2001-03-23


In [470]:
with open('AllRatings.pkl', 'wb') as picklefile:
    pickle.dump(AllRatings, picklefile)

In [471]:
ls

AllRatings.pkl                my_film_data_G.pkl
Box-Office Mojo Script.ipynb  my_film_data_PG.pkl
Web-Scraping.ipynb            my_film_data_PG13.pkl
my_film_data.pkl              my_film_data_R.pkl
